In [12]:
# AiiDA環境をロード
from aiida import load_profile

# プロファイルをロード
load_profile("test_aiida")

Profile<uuid='efe3b646acbe4b46b34848999a0e7c25' name='test_aiida'>

In [2]:
# Int ノードをインポート
from aiida.orm import Int

node = Int(2)
node.store()

<Int: uuid: ca79aa1a-ad97-4834-b28d-eab9b3c73b80 (pk: 144) value: 2>

In [3]:
from aiida.engine import calcfunction


@calcfunction
def multiply(x, y):
    return x * y

In [4]:
x = Int(3)
y = Int(5)
multiply(x, y)

<Int: uuid: 0fe1457f-67f6-4b03-b438-e71040e88489 (pk: 148) value: 15>

In [13]:
from aiida.orm import load_code, load_node, load_group, Dict
from aiida.plugins import DataFactory

code = load_code('pw@localhost')
builder = code.get_builder()

structure = load_node(8)
builder.structure = structure

pseudo_family = load_group('SSSP/1.3/PBE/efficiency')
pseudos = pseudo_family.get_pseudos(structure=structure)
builder.pseudos = pseudos

parameters = {
  'CONTROL': {
    'calculation': 'scf',  # self-consistent field
  },
  'SYSTEM': {
    'ecutwfc': 30.,  # wave function cutoff in Ry
    'ecutrho': 240.,  # density cutoff in Ry
  },
}
builder.parameters = Dict(parameters)

KpointsData = DataFactory('core.array.kpoints')
kpoints = KpointsData()
kpoints.set_kpoints_mesh([4,4,4])
builder.kpoints = kpoints

builder.metadata.options.resources = {'num_machines': 1}
builder.metadata.options.queue_name = "GroupC"


In [14]:
from aiida.engine import submit, run
calcjob_node = run(builder)

/home/hirotosaito/miniconda3/envs/mae/lib/python3.12/site-packages/aiida/storage/psql_dos/orm/nodes.py:224: SAWarning: Identity map already had an identity for (<class 'aiida.storage.sqlite_zip.models.DbNode'>, (172,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  session.commit()
01/29/2025 07:04:42 PM <1456420> aiida.schedulers.plugins.pbsbaseclasses: [ERROR] There are lines without equals sign! [' case \\"$-\\" in ', ' *v*x*)', ' ;; *v*)', ' ;; *x*)', ' ;; *)', ' ;; esac; fi; unset _mlre _mlIFS; if [ -n \\"${IFS+x}\\" ]; then', ' if [ -n \\"`eval \\\'echo ${\\\'$_mlv\\\'+x}\\\'`\\" ]; then', ' eval `eval ${_mlre} /usr/bin/tclsh /usr/share/Modules/libexec/modulecmd.tcl bash \\\'\\"$@\\"\\\'`; else', ' unset IFS; fi; unset _mlre _mlv _mlrv _mlIFS; if [ -n \\"${_mlshdbg:-}\\" ]; then', ' echo \\"Switching to Modules $swname version\\"; source /usr/share/Modules/init/bash; else', ' eval \\"module $@\\"

In [18]:
node = load_node(179)
d = node.get_dict()
d['energy']

-310.56907003575